In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, precision_score

In [2]:
# import csv
# path = 'physiobank_tool/ecgiddb/per_person'
# output_path = 'physiobank_tool/ecgiddb/per_person_output'
# files = []
# with open(f'{path}/header.txt') as f:
#     reader = csv.reader(f)
#     for row in reader:
#         files.append(row[0])
# # files.pop(-1)
# # files.pop(-1)
# # files.pop(13)
# # files.pop(2)
# print(files)


In [3]:
# features_df = pd.DataFrame()
# final_df = pd.DataFrame()

# for file in files:    
#     df = pd.read_csv(f'{output_path}/{file}.csv')
#     df = df.drop(df.columns[0], axis=1)
#     #df['person'] = 'person_' + str(file)
#     df['person'] = str(file)
#     features_df = pd.concat([df, features_df], ignore_index=True)

# # drivedb_final1.csv
# features_df.to_csv('ecgiddb_final_test.csv', index=False)
# #features_df.drop(['qrs_interval.1'], axis=1, inplace=True)

In [5]:
features_df = pd.read_csv('challange_data/block1000.csv')
#features_df.drop(['qrs_interval.1'], axis=1, inplace=True)
features_df.head()
#print(features_df['person'].value_counts())

,mean_q,mean_r,mean_s,mean_p,mean_t,stdev_q,stdev_r,stdev_s,mean_rr_interval,mean_rq_amplitude,mean_qrs_interval,mean_qs_distance,mean_qt_distance,mean_qrs_offset,mean_qrs_onset,person
0,-0.758271,0.424880,-0.115156,0.424880,0.424880,0.000000,0.000000,0.000000,NaN,1.183152,48.0,35.0,12.0,-0.106525,0.141204,tr13-0638
1,-0.469886,0.388483,-0.345001,0.388483,0.391162,0.263695,0.038265,0.199686,511.666667,0.858369,107.5,94.5,41.5,-0.307420,-0.266696,tr13-0638
2,-0.706730,0.465590,-0.091165,0.465590,0.465590,0.058935,0.089288,0.014919,1165.000000,1.172320,86.0,73.0,12.0,-0.090837,0.238561,tr13-0638
3,-0.737378,0.445347,-0.133931,0.445347,0.445347,0.013977,0.002448,0.002395,1205.000000,1.182725,48.0,33.0,12.0,-0.119522,0.181281,tr13-0638
4,-0.697479,0.442879,-0.136184,0.442879,0.442879,0.000000,0.000000,0.000000,NaN,1.140359,49.0,35.0,12.0,-0.114810,0.209274,tr13-0638


In [ ]:
# features_df = pd.read_csv('drivedb_final_test.csv')
# features_df2 = pd.read_csv('mitdb_final_test.csv')
# features_df2['person'] = features_df2.person.apply(str)
# features_df3 = pd.read_csv("ecgiddb_final_test.csv")

# features_df = pd.concat([features_df, features_df2, features_df3], ignore_index=True)

# features_df.drop(['qrs_interval.1'], axis=1, inplace=True)



In [ ]:
# #print(features_df['person'].value_counts())
# features_df = features_df[features_df['person'] != 'person234']
# features_df.head(7)

## Contagem de segmentos duplicados

In [6]:
#ecg_id_features.plot(figsize=(20,12))
print('total original:      ',len(features_df))
print('duplicados no original: ',sum(features_df.duplicated()))
print('número de pessoas únicas no original: ', features_df['person'].nunique())
features_df.drop_duplicates(keep='first', inplace=True)
print('total sem duplicados:      ',len(features_df))
print('duplicados no final: ',sum(features_df.duplicated()))
print('número de pessoas únicas sem dupicados: ', features_df['person'].nunique())

total original:       120000
duplicados no original:  5671
número de pessoas únicas no original:  1000
total sem duplicados:       114329
duplicados no final:  0
número de pessoas únicas sem dupicados:  1000


## Uma função para remover outliers do dataset

In [7]:
def remove_outliers(original_dataset, lower_threshold, upper_threshold, column_names=[]):
    ''' Remove outliers from a dataframe
        Everything above or below these percentiles will be cut off
    '''
    # TODO: add treatment for not numerical columns
    dataset = original_dataset.copy()
    
    if column_names:
        for column in column_names:
            removed_outliers = remove(dataset[column], lower_threshold, upper_threshold)
            # save the indexes of rows that must be removed
            indexes_for_removal = dataset[column][~removed_outliers].index
            # in fact remove outliers from this column 
            #print(indexes_for_removal)
            dataset.drop(indexes_for_removal, inplace=True)
            #print(f'removed {len(indexes_for_removal)} outliers for column {column}')
            #print(f'remaining itens in dataset: {len(dataset)}')
        return dataset
            
    else:
        column_names = list(dataset.columns)
        for column in column_names:
            removed_outliers = remove(dataset[column], lower_threshold, upper_threshold)
            # save the indexes of rows that must be removed
            indexes_for_removal = dataset[column][~removed_outliers].index
            # in fact remove outliers from this column 
            dataset.drop(indexes_for_removal, inplace=True)
        return dataset
    
def remove(series, lower_threshold, upper_threshold):
    ''' Remove outliers from a single pandas Series '''
    # create a boolean mask where False values are the outliers
    removed_outliers = series.between(series.quantile(lower_threshold),
                                      series.quantile(upper_threshold))
    return removed_outliers

## Testando com um classificador *random forests*

### LabelEnconder, caso necessário

In [ ]:
# caso precisarmos de um label enconder
#le = LabelEncoder()
#ecg_id_features['person'] = le.fit_transform(ecg_id_features['person'])

### Remoção de outliers do rr_interval (feature mais caótica)

In [8]:
lower_threshold = 0.005 # 0,5%
upper_threshold = 0.995 # 0,5%

#lower_threshold = 0.01 # 1%
#upper_threshold = 0.99 # 1%

column_names = list(features_df.columns)
person_index = column_names.index('person')
column_names.pop(person_index)
print(f'Original features had {len(features_df)} rows')

cleaned_features_df = remove_outliers(features_df,lower_threshold, upper_threshold, column_names=column_names)

print(f'Features without outliers has {len(cleaned_features_df)} rows')

#cleaned_features_df.to_csv('driver_db_cleaned_features.csv', index=False)

Original features had 114329 rows
Features without outliers has 93378 rows


In [9]:
# Remove os rótulos
X = cleaned_features_df.drop(['person'], axis=1)
# Preenche os dados faltosos com a média da respectiva coluna
X = X.apply(lambda x: x.fillna(x.mean()))
# O objeto da predição é 'y' (os rótulos das classes)
y = cleaned_features_df['person']

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

scaler = StandardScaler()
train_X = scaler.fit_transform(X_train)
test_X = scaler.transform(X_test)


size_of_chunk = 10
list_of_training_x = np.array_split(X_train, 10)
list_of_training_y = np.array_split(y_train, 10)


estimators=20
rf = RandomForestClassifier(n_estimators=estimators, max_depth=100, min_samples_leaf=3, min_samples_split=10, verbose=True)


for i in range(10):
    rf.fit(list_of_training_x[i], list_of_training_y[i])
    
    rf.set_params(n_estimators=estimators)
    estimators+=20

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    7.7s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    6.5s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:   15.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:   25.5s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  80 out of  80 | elapsed:   27.8s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   35.6s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_j

### Separar os dados em porções de treino e teste

In [11]:
# Remove os rótulos
X = cleaned_features_df.drop(['person'], axis=1)
# Preenche os dados faltosos com a média da respectiva coluna
X = X.apply(lambda x: x.fillna(x.mean()))
# O objeto da predição é 'y' (os rótulos das classes)
y = cleaned_features_df['person']

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

## Realizar Standardization nos dados

In [13]:
chunks = np.array_split(cleaned_features_df, 3)
print(type(chunks[0]))
for chunk in chunks:
    for X, y in 

SyntaxError: invalid syntax (<ipython-input-13-b78a8e815016>, line 4)

## Dá uma olhada nos dados aṕos filtragem e Standardzation

In [ ]:
#X.plot(figsize=(12,8))

## Alimenta o classificador com os dados

In [ ]:
chunks = np.array_split(df, 3)
print(type(chunks))

In [ ]:
# os parametros passados ao classificador são os utilizados
# previamente, obtidos pelo GridSearch
rf = RandomForestClassifier(n_jobs=4, n_estimators=200, max_depth=100, min_samples_leaf=3, min_samples_split=10, verbose=True)
#print(y_train)
rf.fit(X_train, y_train)

In [ ]:
predictions = rf.predict(X_test)

In [ ]:
classification_output = classification_report(y_test, predictions)
print(classification_output)
##f = open("demofile.txt", "w")
#f.write(classification_output)
#f.close()

In [ ]:
confusion_matrix(y_test, predictions)

## Verificando a importância das features

In [ ]:
columns_names = list(features_df.columns)
importances = [(feature, round(importance, 2)) for feature, importance in zip(columns_names, rf.feature_importances_)]
sorted_importances = sorted(importances, key=lambda x:x[1], reverse=True) 

for pair in sorted_importances:
    print('Feature: {:20} Importance: {}'.format(*pair))

In [ ]:
x_values = range(len(rf.feature_importances_))
plt.figure(figsize=(16,7))
plt.ylabel('Importance')
plt.xlabel('Feature')
plt.title('Feature Importance')
plt.xticks(x_values, columns_names, rotation='vertical')
plt.bar(x_values, rf.feature_importances_, orientation = 'vertical')

## Visualizando os motoristas com as menores acurácias

In [ ]:
def plot_driver_info(driver_info):
    # driver_info is a dataset

    # Cria 2 subplots
    fig, axs = plt.subplots(1,2, figsize=(21,8))

    # Visualização Geral
    driver_info.plot(ax=axs[0])
    plt.title("Drive 05",fontsize=20)

    # Boxplot com possíveis outliers
    driver_info.plot(kind='box', ax=axs[1])
    plt.title("'Pontos fora da curva' saltando para fora das caixas")

In [ ]:
# para o driver 05
plot_driver_info(cleaned_features_df[cleaned_features_df['person'] == 'drive05'])

In [ ]:
# para o driver 01 - ALTA PRECISÃO
#plot_driver_info(ecg_id_features[ecg_id_features['person'] == 'drive02'])

In [ ]:
plot_driver_info(features_df[features_df['person'] == 'drive07'])

In [ ]:
plot_driver_info(cleaned_features_df[cleaned_features_df['person'] == 'drive09'])

In [ ]:
plot_driver_info(cleaned_features_df[cleaned_features_df['person'] == 'drive11'])

In [ ]:
Remove os rótulos
X = features_df.drop(['person'], axis=1)
Preenche os dados faltosos com a média da respectiva coluna
X = X.apply(lambda x: x.fillna(x.mean()))
O objeto da predição é 'y' (os rótulos das classes)
y = features_df['person']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

from sklearn.model_selection import cross_val_score
scores = cross_val_score(rf, X, y, cv=10)

In [ ]:
scores.mean()

In [ ]:
lista = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
np.split(lista, 5)